<img src="https://github.com/LinkedEarth/Logos/blob/master/PyLiPD/pyLiPD_logo1_transparent.png?raw=true" width ="800">

# Creating LiPD files from a tabular template

## Authors

[Deborah Khider](https://orcid.org/0000-0001-7501-8430)


## Preamble

If you are planning to only create one LiPD file on your own, we recommend using the [LiPD Playground](https://lipd.net/playground). This tutorial is intended for users who wish to programatically create multiple files from a template. 

In this example, we use [this templated file](https://github.com/LinkedEarth/pylipdTutorials/blob/main/data/Oman.Tian.2023.xlsx). You can repurpose the Excel template as needed; it is only meant as an example. 

### Goals

* Create a LiPD formatted Dataset from an excel template
* Adding an ensemble table 
* Save the Dataset to a file

Reading Time: 20 minutes

### Keywords

LiPD, LinkedEarth Ontology, Object-Oriented Programming

### Pre-requisites

An understanding of OOP and the LinkedEarth Ontology. Completion of [Dataset class example](L3_dataset_class.ipynb). An understanding how to [edit LiPD files](L3_editing.ipynb) can also be useful.

For reference, below is a diagram of the classed in PyliPD, the methods associated with them and the resulting objects:

![image](https://github.com/LinkedEarth/pylipd/blob/main/examples/notebooks/UMLDiagram.png?raw=true)

## Data Description

- Tian, Y., Fleitmann, D., Zhang, Q., Sha, L., Wassenburg, J. A., Axelsson, J., … Cheng, H. (2023). Holocene climate change in southern Oman deciphered by speleothem records and climate model simulations. Nature Communications, 14(1), 4718. doi:[10.1038/s41467-023-40454-z](https://www.nature.com/articles/s41467-023-40454-z). 

## Demonstration

Let's import the necessary packages. 

In [1]:
from pylipd.classes.dataset import Dataset
from pylipd.classes.archivetype import ArchiveTypeConstants
from pylipd.classes.funding import Funding
from pylipd.classes.interpretation import Interpretation
from pylipd.classes.interpretationvariable import InterpretationVariableConstants
from pylipd.classes.location import Location
from pylipd.classes.paleodata import PaleoData
from pylipd.classes.datatable import DataTable
from pylipd.classes.paleounit import PaleoUnitConstants
from pylipd.classes.paleovariable import PaleoVariableConstants
from pylipd.classes.person import Person
from pylipd.classes.publication import Publication
from pylipd.classes.resolution import Resolution
from pylipd.classes.variable import Variable
from pylipd.classes.model import Model
from pylipd.classes.chrondata import ChronData

from pylipd import LiPD

import pandas as pd
import json
import numpy as np

import re

### Opening our template file

The Excel file contains the following sheets:
- `About`
- `Guidelines`
- `Metadata`
- `paleo1measurementTable1`
- `chron1measurementTable1`
- `Lists`

The information we are interested in contained in `Metadata`, `paleo1measurementTable1` and `chron1measurementTable1`. Notice that the last two sheets follow the [LiPD nomenclature](https://lipd.net/playground) closely and this can be helpful to keep track of the tables and where to insert them. However, you may choose any names that is convenient for you. 

Let's start with the root metadata portion.

### Metadata

If you have a look at the Metadata sheet in the Excel file, you should notice that the information is orgnaized in four sections:
* root metadata, which contains general information about the dataset such as its name and the type of archive the measurements were made on (e.g., coral, speleothem).
* Publication information - Note that if more than one publication is associated with the dataset, then this information can be added in seperate columns. 
* Location information
* Funding information

The first step is to create a function to separate these different sections into four dataframes:

In [2]:
def read_metadata(df):
    '''
    Reads the inforamtion contained in the metadata sheet of the Excel template.
    Note that the algorithm uses the blank lines in the template to denote the block (e.g., publication).

    The code returns 4 pieces of information: root meatadata, location metadata, funding metadata, and publication metadata.
    '''
    # Check for empty rows across all columns
    empty_rows = df.isnull().all(axis=1)
    
    # Initialize the start index of the first table
    start_idx = 0
    tables = []
    
    # Iterate through the indices of the DataFrame
    for idx in empty_rows[empty_rows].index:
        # Slice from the current start index to the row before the empty row
        if not df[start_idx:idx].empty:
            current_table = df[start_idx:idx]
            # Check if the table should use its first row as header
            if start_idx != 0:  # Skip header adjustment for the first table
                current_table.columns = current_table.iloc[0]  # Set first row as header
                current_table = current_table[1:]  # Remove the first row from the data
                current_table.reset_index(drop=True, inplace=True)  # Reset index after dropping row
            tables.append(current_table)
        # Update start_idx to the row after the current empty row
        start_idx = idx + 1
    
    # Handle the last table, if any, after the last empty row to the end of the DataFrame
    if start_idx < len(df):
        current_table = df[start_idx:]
        if start_idx != 0:  # Likely unnecessary check but for consistency
            current_table.columns = current_table.iloc[0]  # Set first row as header
            current_table = current_table[1:]  # Remove the first row from the data
            current_table.reset_index(drop=True, inplace=True)
        tables.append(current_table)

    # place the tables according to their types
    root=tables[0]
    pub=tables[1]
    geo=tables[2]
    fund=tables[3]

    return root, pub, geo, fund    

In [3]:
file_path = "../data/Oman.Tian.2023.xlsx"
sheet_name = 'Metadata'

df = pd.read_excel(file_path, sheet_name=sheet_name)

In [4]:
# get the various tables
root, pub, geo, fund = read_metadata(df)

The next step is to create an empty [`Dataset`](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.dataset.Dataset) object so we can start storing the information. 

In [5]:
ds = Dataset()

Let's go over each of the information that we have stored in Pandas Dataframe, namely some root information such as the name of the dataset, geographical location, information about the source of funding, and publication(s) associated with the data. 

Let's start with the root information:

In [6]:
root

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Dataset Name (siteName.firstAuthor.year),Oman.Tian.2023,NaN
1,Archive Type,speleothem,NaN
2,Original Source_URL (if applicable),https://www.nature.com/articles/s41467-023-404...,NaN
3,"Investigators (Lastname, first; lastname2, fir...","Tian, Y., Fleitmann, D., Zhang, Q., Sha, L. J,...",NaN


Let's add this information to the `Dataset` object. You can see a list of possible properties under the [`Dataset` class in the documentation](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.dataset.Dataset). Remember from our previous tutorial that `set+PropertyName` is meant to create the information, which is what we will be doing here. If you cannot find you property in the list, don't panic! The LiPD format is flexible so you can add your own properties using the [`set_non_standard_property(key,value)` function](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.dataset.Dataset.set_non_standard_property). 

In [7]:
ds.setName(root.iloc[0,1]) # set the name for the dataset
ds.setArchiveType(root.iloc[1,1]) #set the archive type
ds.setOriginalDataUrl(root.iloc[2,1]) # set the original data URL

The next step is to enter the investigators, which takes a list of `Person` objects (see Figure in the Preamble): 

In [8]:
authors = root.iloc[3,1]

# Step 1: Split the string by commas
parts = authors.split(',')

# Prepare a list to hold the formatted names
investigators = []

# Step 2: Iterate over the parts to process each
for i in range(0, len(parts) - 1, 2):  # Step by 2 since each name and initial are next to each other
    last_name = parts[i].strip()  # Remove any leading/trailing whitespace
    initial = parts[i + 1].strip()  # The initial follows the last name
    person = Person() # create the Person object
    person.setName(f"{last_name}, {initial}")
    investigators.append(person)

# Step 3: Store the list of Persons into the ds object
ds.setInvestigators(investigators)

Let's have a quick look at what we've done so far:

In [9]:
ds.getArchiveType()

'speleothem'

In [10]:
ds.getName()

'Oman.Tian.2023'

In [11]:
ds.getInvestigators()[0].getName() #get the name of the first person in the list

'Tian, Y.'

Everything looks good, let's move on to publication information:

In [12]:
pub

5,Publication Section,Ref #1,Ref #2
0,"Authors (last, first; last2, first2; separate ...","Tian, Y., Fleitmann, D., Zhang, Q., Sha, L. J,...",NaN
1,Publication title,Holocene climate change in southern Oman decip...,NaN
2,Journal,Nature Communications,NaN
3,Year,2023,NaN
4,Volume,NaN,NaN
5,Issue,NaN,NaN
6,Pages,NaN,NaN
7,Report Number,NaN,NaN
8,DOI,10.1038/s41467-023-40454-z,NaN
9,Abstract,Qunf Cave oxygen isotope (δ18Oc) record from s...,NaN


The first step is to create a [`Publication` object](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.publication.Publication). In this case, we only have one publication; otherwise you may need to loop over the various columns in the dataframe to create others. 

In [13]:
pub1 = Publication()

And now let's add the information. Let's start with the authors:

In [14]:
authors = pub.iloc[0,1]

# Step 1: Split the string by commas
parts = authors.split(',')

# Prepare a list to hold the formatted names
investigators = []

# Step 2: Iterate over the parts to process each
for i in range(0, len(parts) - 1, 2):  # Step by 2 since each name and initial are next to each other
    last_name = parts[i].strip()  # Remove any leading/trailing whitespace
    initial = parts[i + 1].strip()  # The initial follows the last name
    person = Person() # create the Person object
    person.setName(f"{last_name}, {initial}")
    investigators.append(person)

# Step 3: Store the list of Persons into the ds object
pub1.setAuthors(investigators)

Let's add other information:

In [15]:
pub1.setTitle(pub.iloc[1,1])
pub1.setJournal(pub.iloc[2,1])
pub1.setYear(pub.iloc[3,1])
pub1.setDOI(pub.iloc[8,1])
pub1.setAbstract(pub.iloc[9,1])

Let's add our `Publication` object to the `Dataset` object:

In [16]:
ds.addPublication(pub1)

Let's add geographical information next. First, we need to create a [`Location` object](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.location.Location):

In [17]:
loc = Location()

In [18]:
geo

18,Site Information,Use appropriate significant digits for all values,NaN
0,"Northernmost latitude (decimal degree, South n...",17.17,NaN
1,"Southernmost latitude (decimal degree, South n...",17.17,NaN
2,"Easternmost longitude (decimal degree, West ne...",54.3,NaN
3,"Westernmost longitude (decimal degree, West ne...",54.3,NaN
4,"elevation (m), below sea level negative",650,NaN


In [19]:
loc.setLatitude(geo.iloc[0,1])
loc.setLongitude(geo.iloc[2,1])
loc.setElevation(geo.iloc[4,1])

Let's add the `Location` object into the `Dataset`:

In [20]:
ds.setLocation(loc)

Finally, let's look at funding information:

In [21]:
fund

26,Funding_Agency,"Any additional Funding agencies and grants should be entered in Columns C,D, etc.",NaN
0,Funding_Agency_Name,NaN,NaN
1,Grant,NaN,NaN
2,Principal_Investigator,NaN,NaN
3,country,NaN,NaN


Since no information is available, let's move on to the PaleoData Section.

### PaleoData

In [22]:
sheet_name = 'paleo1measurementTable1'

# Read the information into a Pandas DataFrame
pd_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Drop completely empty rows
pd_df = pd_df.dropna(how="all").reset_index(drop=True)

Let's create a [`PaleoData` object](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.paleodata.PaleoData):

In [23]:
paleodata = PaleoData()

Our next step is to create measurement tables in this object. To do so, we can use the [`DataTable` object](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.datatable.DataTable): 

In [24]:
table = DataTable()

Now let's add some information about the table such as the name and the value use for missin values in the data:

In [25]:
table.setFileName("paleo0measurement0.csv")
table.setMissingValue("NaN")

The next step is to add columns to our table. In other words, we need to create some variables.

Let's have a look at the sheet in the excel file. It contains three sections:
* Notes, which would be attached to the table
* Variables information such as the name, units... Each row in the template represents metadata information for each of the column.
* Data, which contains the numerical values for the variables.

Let's first create an algorithm that separates the various section and returns the information in three dataframes (Notes, Variables, and Data).

In [26]:
def extract_data(df): 
    '''
    This function extracts the relevant sections for measurementTables. 
    '''
    # Find the index positions of the section headers
    notes_start = df[df[0] == "Notes"].index[0]
    variables_start = df[df[0] == "Variables"].index[0]
    data_start = df[df[0] == "Data"].index[0]

    # Extract sections, ensuring blank rows are removed
    df_notes = df.iloc[notes_start + 1:variables_start].dropna(how="all").reset_index(drop=True)

    # Extract the Variables section
    df_variables_raw = df.iloc[variables_start + 1:data_start].dropna(how="all").reset_index(drop=True)

    # Set the first row as the header for the Variables section
    df_variables = df_variables_raw[1:].reset_index(drop=True)  # Data rows
    df_variables.columns = df_variables_raw.iloc[0]  # Set first row as column headers

    # Extract the Data section
    df_data_raw = df.iloc[data_start + 2:].dropna(how="all").reset_index(drop=True)

    # Correctly skip the first row and set the second row as the header
    df_data = df_data_raw.iloc[1:].reset_index(drop=True)  # Skip first row, keep rest
    df_data.columns = df_data_raw.iloc[0]  # Use second row as column headers
    df_data = df_data.dropna(axis=1, how="all") # Drop the columns with NaN.

    return df_notes, df_variables, df_data

In [27]:
df_notes, df_variables, df_data = extract_data(pd_df)

Let's loop over the variables and get the relevant information. We will also be calculating some relevant information such as the average value and average resolution.

In LiPD, each variable is also given a unique ID. The function below generates one: 

In [28]:
import uuid

def generate_unique_id(prefix='PYD'):
    # Generate a random UUID
    random_uuid = uuid.uuid4()  # Generates a random UUID.
    
    # Convert UUID format to the specific format we need
    # UUID is usually in the form '1e2a2846-2048-480b-9ec6-674daef472bd' so we slice and insert accordingly
    id_str = str(random_uuid)
    formatted_id = f"{prefix}-{id_str[:5]}-{id_str[9:13]}-{id_str[14:18]}-{id_str[19:23]}-{id_str[24:28]}"
    
    return formatted_id

In [29]:
variables = []

res = df_data.iloc[:, 1].diff()[1:].to_numpy()
Res = Resolution() # create a Resolution object - it will be the same for all variables since it is based on time
Res.setMinValue(np.min(res))
Res.setMaxValue(np.max(res))
Res.setMeanValue(np.mean(res))
Res.setMedianValue(np.median(res))

for index, row in df_variables.iterrows():
    var = Variable()
    var.setName(row['variableName']) # name of the variable
    var.setColumnNumber(index+1) #The column in which the data is stored. Note that LiPD uses index 1
    var.setVariableId(generate_unique_id(prefix='TIAN')) # create a unique ID for the variable
    var.setUnits(row['Units'])
    # Make sure the data is JSON writable (no numpy arrays or Pandas DataFrame)
    var.setValues(json.dumps(df_data.iloc[:,index].tolist()))
    # Calculate some metadata about the values - this makes it easier to do some queries later on, including looking for data in a particular time slice. 
    var.setMinValue(df_data.iloc[:,index].min())
    var.setMaxValue(df_data.iloc[:,index].max())
    var.setMeanValue(df_data.iloc[:,index].mean())
    var.setMedianValue(df_data.iloc[:,index].median())
    # Attach the resolution metadata information to the variable
    var.setResolution(Res)
    # append in the list
    variables.append(var)                    

Let's now put our variables in the `DataTable`:

In [30]:
table.setVariables(variables)

The `Table` into the `PaleoData` object:

In [31]:
paleodata.setMeasurementTables([table])

And finally, the `PaleoData` object into the `Dataset`:

In [32]:
ds.addPaleoData(paleodata)

### ChronData

The next step is to create a ChronData object to store the information about chronology. In the last section, we used an OOP approach to add the information about each variable. In this section, we will use an approach involving `Pandas DataFrame`.

Let's open the data. The same function we wrote to read in the PaleoData can be used here since the template is the same for both objects.

In [33]:
sheet_name = 'chron1measurementTable1'

# Read the information into a Pandas DataFrame
cd_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Drop completely empty rows
cd_df = cd_df.dropna(how="all").reset_index(drop=True)

Let's create a [`ChronData` object](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.chrondata.ChronData):

In [34]:
chrondata = ChronData()

We need to create a `DataTable`, a process similar to what we have done for the PaleoData:

In [35]:
chrontable = DataTable()

Let's add some basic information about the table:

In [36]:
chrontable.setFileName("chron0measurement0.csv")
chrontable.setMissingValue("NaN")

In [37]:
dfc_notes, dfc_variables, dfc_data = extract_data(cd_df)

We will used the [`setDataFrame` function](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.datatable.DataTable.setDataFrame) to incorporate the columns into the table. In this framework, the values are held in a dataframe (which is represented by `dfc_data` in our framework) and the metadata for each variable is added as attributes to the DataFrame. 

In [38]:
metadata_dict = {} # create a dictionary

for index, row in dfc_variables.iterrows():
    temp_dict = {}
    temp_dict['number']=index+1
    temp_dict['variableName']=row['variableName']
    temp_dict['TSid']= generate_unique_id(prefix='TC')
    if pd.notna(row['Units']):
        temp_dict['units']=row['Units']
    else:
        temp_dict['units']='NA'
    try:
        temp_dict['hasMinValue']=dfc_data.iloc[:,index].min()
        temp_dict['hasMaxValue']=dfc_data.iloc[:,index].max()
        temp_dict['hasMeanValue']=dfc_data.iloc[:,index].mean()
        temp_dict['hasMedianValue']=dfc_data.iloc[:,index].median()
    except:
        pass      
    metadata_dict[row['variableName']]=temp_dict

dfc_data.attrs = metadata_dict

Use the DataFrame to construct the `Table` object:

In [39]:
chrontable.setDataFrame(dfc_data)

Put the `Table` into the `ChronData` object:

In [40]:
chrondata.setMeasurementTables([chrontable])

Put the `ChronData` into the `Dataset`:

In [41]:
ds.addChronData(chrondata)

#### Adding an ensemble table

This particular dataset also has an ensemble table available in [Oman.Tian.2023.chrondf.csv](https://github.com/LinkedEarth/pylipdTutorials/raw/refs/heads/main/data/Oman.Tian.2023.chrondf.csv). Let's add the information to an ensembleTable in the LiPD file.

Let's first open the data: 

In [42]:
ens_path = "../data/Oman.Tian.2023.chrondf.csv"

df_ens = pd.read_csv(ens_path)
df_ens.head()

,Unnamed: 0,age,d18O,depth,chron_0,chron_1,chron_2,chron_3,chron_4,chron_5,...,chron_990,chron_991,chron_992,chron_993,chron_994,chron_995,chron_996,chron_997,chron_998,chron_999
0,0,401.88,-0.20,0.00,444.0,445.0,417.0,290.0,403.0,349.0,...,430.0,407.0,410.0,369.0,392.0,347.0,359.0,416.0,414.0,408.0
1,1,408.55,-0.59,0.69,445.0,446.0,445.0,355.0,404.0,367.0,...,431.0,410.0,411.0,371.0,395.0,358.0,362.0,417.0,416.0,414.0
2,2,424.07,-0.58,1.37,489.0,491.0,482.0,425.0,448.0,422.0,...,469.0,468.0,459.0,411.0,437.0,397.0,422.0,441.0,463.0,435.0
3,3,438.75,-0.73,2.06,562.0,568.0,525.0,495.0,521.0,504.0,...,533.0,565.0,541.0,479.0,507.0,455.0,524.0,482.0,541.0,466.0
4,4,450.24,-1.26,2.75,568.0,574.0,536.0,503.0,531.0,514.0,...,547.0,575.0,548.0,491.0,510.0,461.0,530.0,493.0,549.0,480.0


<div style="color: white; background-color: #F9ACAA; border-left: 6px solid #CC0000; padding: 0.5em;">
    &#9888; <strong>Warning:</strong> LiPD files require ensemble tables to have the following format: the first column contains depth and the other column the ensemble members as a list. 
</div>

Our first step is to drop the first 3 columns in the DataFrame:

In [43]:
df_ens = df_ens.iloc[:, 3:]
df_ens.head()

,depth,chron_0,chron_1,chron_2,chron_3,chron_4,chron_5,chron_6,chron_7,chron_8,...,chron_990,chron_991,chron_992,chron_993,chron_994,chron_995,chron_996,chron_997,chron_998,chron_999
0,0.00,444.0,445.0,417.0,290.0,403.0,349.0,387.0,412.0,341.0,...,430.0,407.0,410.0,369.0,392.0,347.0,359.0,416.0,414.0,408.0
1,0.69,445.0,446.0,445.0,355.0,404.0,367.0,405.0,413.0,369.0,...,431.0,410.0,411.0,371.0,395.0,358.0,362.0,417.0,416.0,414.0
2,1.37,489.0,491.0,482.0,425.0,448.0,422.0,447.0,442.0,441.0,...,469.0,468.0,459.0,411.0,437.0,397.0,422.0,441.0,463.0,435.0
3,2.06,562.0,568.0,525.0,495.0,521.0,504.0,506.0,492.0,543.0,...,533.0,565.0,541.0,479.0,507.0,455.0,524.0,482.0,541.0,466.0
4,2.75,568.0,574.0,536.0,503.0,531.0,514.0,511.0,501.0,555.0,...,547.0,575.0,548.0,491.0,510.0,461.0,530.0,493.0,549.0,480.0


Next, let's create the proper dataframe format. The first column will stay the same. The second column will contain each values on the ensemble in a list: 

In [44]:
#Let's keep the first column (depth) in place
ens_table = pd.DataFrame({'depth': df_ens['depth'].tolist()})

# Add the year data - each row will contain one vector from a data array. 
array = df_ens.iloc[:, 1:].to_numpy()
ens_table['year'] = [array[i,:].tolist() for i in range(array.shape[0])]
ens_table.head()

,depth,year
0,0.00,"[444.0, 445.0, 417.0, 290.0, 403.0, 349.0, 387..."
1,0.69,"[445.0, 446.0, 445.0, 355.0, 404.0, 367.0, 405..."
2,1.37,"[489.0, 491.0, 482.0, 425.0, 448.0, 422.0, 447..."
3,2.06,"[562.0, 568.0, 525.0, 495.0, 521.0, 504.0, 506..."
4,2.75,"[568.0, 574.0, 536.0, 503.0, 531.0, 514.0, 511..."


Add attributes to the Pandas Dataframe to store the metadata. 

<div style="color: white; background-color: #F9ACAA; border-left: 6px solid #CC0000; padding: 0.5em;">
    &#9888; <strong>Warning:</strong> Metadata attributes are necessary to save a LiPD file. 
</div>

In [45]:
num_year_columns = len(array[0,:])
year_columns = [i+2 for i in range(num_year_columns)]
ens_table.attrs = {
    'year': {'number': str(year_columns), 'variableName': 'year', 'units': 'yr AD', 'TSid':generate_unique_id()},
    'depth': {'number': 1, 'variableName': 'depth', 'units': 'cm', 'TSid':generate_unique_id()}
}

Let's create a `DataTable` object for our ensemble table:

In [46]:
ensemble_table = DataTable()

In [47]:
ensemble_table.setDataFrame(ens_table)
ensemble_table.setFileName("chron0model0ensemble0.csv")

Now add the table to a model:

In [48]:
model = Model()
model.addEnsembleTable(ensemble_table)

And add the Model to a ChronData object:

In [49]:
chrondata.addModeledBy(model)

### Writing a LiPD file

The last step in this process is to write to a LiPD file. To do so, you need to pass the Dataset `ds` back into a LiPD object:

In [50]:
lipd = LiPD()
lipd.load_datasets([ds])
lipd.create_lipd(ds.getName(), "../data/Oman.Tian.2023.lpd");

### Opening the LiPD file

Let's re-open the LiPD file that we have just created and check some of our work.

In [51]:
L = LiPD()
file = "../data/Oman.Tian.2023.lpd"

L.load_datasets(file)

AttributeError: 'str' object has no attribute 'getPaleoData'